In [1]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from tensorflow.keras import backend as K
from imutils import paths

import gzip
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

from rl_with_videos.preprocessors.convnet import convnet_preprocessor

C:\Users\irohc\anaconda3\envs\drl_proj\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\irohc\anaconda3\envs\drl_proj\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\irohc\anaconda3\envs\drl_proj\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\irohc\anaconda3\envs\drl_proj\lib\site-packages\tensorf

In [2]:
POSITIVE_PATH = "C:/nyu/DRL/final_project/dataset/acrobot-975-1000.pkl"
NEGATIVE_PATH = "C:/nyu/DRL/final_project/dataset/acrobot-25-50.pkl"

MAX_SEQ_LENGTH = 40

In [3]:
def load_sequence(VIDEO_PATH):
    with gzip.open(VIDEO_PATH, 'rb') as f:
        latest_samples = pickle.load(f)

    key = list(latest_samples.keys())[0]
    num_samples = latest_samples[key].shape[0]
    for field_name, data in latest_samples.items():
        assert data.shape[0] == num_samples, data.shape
    
    return latest_samples

In [4]:
p_sample = load_sequence(POSITIVE_PATH)
n_sample = load_sequence(NEGATIVE_PATH)

In [5]:
p_size = p_sample['observations'].shape[0]
n_size = n_sample['observations'].shape[0]

In [6]:
def prepare_input(x_sample, x_size):
    input_x = []
    seq = []
    for i in range(0, x_size):
        seq.append(x_sample['observations'][i])
        if(x_sample['terminals'][i]):
            seq = np.array(seq)
            select_frame = np.linspace(0, seq.shape[0]-1, MAX_SEQ_LENGTH,endpoint=True,retstep=True,dtype=int)[0]
            input_x.append(seq[select_frame])
            seq = []
    return np.array(input_x)

def merge_input(p_input, n_input):
    input_batch = []
    label = []
    for i in range(p_input.shape[0]):
        input_batch.append(p_input[i])
        label.append(1)
    for i in range(n_input.shape[0]):
        input_batch.append(n_input[i])
        label.append(0)
    
    input_batch = np.array(input_batch).astype('float32')
    label = np.array(label)
    return input_batch, label

In [7]:
p_input = prepare_input(p_sample, p_size)
n_input = prepare_input(n_sample, n_size)

In [8]:
train_x, train_y = merge_input(p_input, n_input)

In [9]:
from rl_with_videos.models.feedforward import feedforward_model

In [10]:
feature_extractor = feedforward_model([(6,)], 6, [6], output_activation = 'relu')
feature_extractor.trainable = True
feature_extractor.summary()

name: feedforward_model
inputs: [<tf.Tensor 'input_1:0' shape=(?, 6) dtype=float32>]
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
lambda (Lambda)              (None, 6)                 0         
_________________________________________________________________
dense (Dense)                (None, 6)                 42        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
Total params: 84
Trainable params: 84
Non-trainable params: 0
_________________________________________________________________


In [11]:
class PositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

#     def compute_mask(self, inputs, mask=None):
#         mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
#         return mask

class MultiHeadAttention(keras.layers.Layer):
    
    def __init__(self,multiheads, head_dim,mask_right=False,**kwargs):
        self.multiheads = multiheads
        self.head_dim = head_dim
        self.output_dim = multiheads * head_dim
        self.mask_right = mask_right
        super(MultiHeadAttention, self).__init__(**kwargs)
        
    def compute_output_shape(self,input_shape):
        return (input_shape[0][0],input_shape[0][1],self.output_dim) #shape=[batch_size,Q_sequence_length,self.multiheads*self.head_dim]

    def build(self,input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1].value, self.output_dim),#input_shape[0] -> Q_seq
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1].value, self.output_dim),#input_shape[1] -> K_seq
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1].value, self.output_dim),#input_shape[2] -> V_seq
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(MultiHeadAttention, self).build(input_shape)
    
    def Mask(self,inputs,seq_len,mode='add'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(indices=seq_len[:,0],num_classes=K.shape(inputs)[1])#mask.shape=[batch_size,short_sequence_length],mask=[[0,0,0,0,1,0,0,..],[0,1,0,0,0,0,0...]...]
            mask = 1 - K.cumsum(mask,axis=1)#mask.shape=[batch_size,short_sequence_length],mask=[[1,1,1,1,0,0,0,...],[1,0,0,0,0,0,0,...]...]
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            elif mode == 'add':
                return inputs - (1 - mask) * 1e12
    
    def call(self,QKVs):

        if len(QKVs) == 3:
            Q_seq,K_seq,V_seq = QKVs
            Q_len,V_len = None,None
        elif len(QKVs) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = QKVs
 
        Q_seq = K.dot(Q_seq,self.WQ)#Q_seq.shape=[batch_size,Q_sequence_length,self.output_dim]=[batch_size,Q_sequence_length,self.multiheads*self.head_dim] 
        Q_seq = K.reshape(Q_seq,shape=(-1,K.shape(Q_seq)[1],self.multiheads,self.head_dim))#Q_seq.shape=[batch_size,Q_sequence_length,self.multiheads,self.head_dim]
        Q_seq = K.permute_dimensions(Q_seq,pattern=(0,2,1,3))#Q_seq.shape=[batch_size,self.multiheads,Q_sequence_length,self.head_dim]

        K_seq = K.dot(K_seq,self.WK)
        K_seq = K.reshape(K_seq,shape=(-1,K.shape(K_seq)[1],self.multiheads,self.head_dim))
        K_seq = K.permute_dimensions(K_seq,pattern=(0,2,1,3))

        V_seq = K.dot(V_seq,self.WV)
        V_seq = K.reshape(V_seq,shape=(-1,K.shape(V_seq)[1],self.multiheads,self.head_dim))
        V_seq = K.permute_dimensions(V_seq,pattern=(0,2,1,3))

        A = K.batch_dot(Q_seq,K_seq,axes=[3,3])/K.sqrt(K.cast(self.head_dim,dtype='float32'))#A.shape=[batch_size,self.multiheads,Q_sequence_length,K_sequence_length]
        A = K.permute_dimensions(A,pattern=(0,3,2,1))#A.shape=[batch_size,K_sequence_length,Q_sequence_length,self.multiheads]

        A = self.Mask(A,V_len,'add')
        A = K.permute_dimensions(A,pattern=(0,3,2,1))#A.shape=[batch_size,self.multiheads,Q_sequence_length,K_sequence_length]
        
        if self.mask_right:
            ones = K.ones_like(A[:1,:1])
            lower_triangular = K.tf.matrix_band_part(ones,num_lower=-1,num_upper=0) 
            mask = (ones - lower_triangular) * 1e12 
            A = A - mask #Element-wise subtract，A.shape=[batch_size,self.multiheads,Q_sequence_length,K_sequence_length]
        A = K.softmax(A) #A.shape=[batch_size,self.multiheads,Q_sequence_length,K_sequence_length]
        #V_seq.shape=[batch_size,V_sequence_length,V_embedding_dim]
        O_seq = K.batch_dot(A,V_seq,axes=[3,2])#O_seq.shape=[batch_size,self.multiheads,Q_sequence_length,V_sequence_length]
        O_seq = K.permute_dimensions(O_seq,pattern=(0,2,1,3))#O_seq.shape=[batch_size,Q_sequence_length,self.multiheads,V_sequence_length]
        O_seq = K.reshape(O_seq,shape=(-1,K.shape(O_seq)[1],self.output_dim))#O_seq.shape=[,Q_sequence_length,self.multiheads*self.head_dim]
        O_seq = self.Mask(O_seq,Q_len,'mul')
        return O_seq

class TransformerEncoder(keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = MultiHeadAttention(
            num_heads, embed_dim
        )
        self.dense_proj = keras.Sequential(
            [keras.layers.Dense(dense_dim, activation='relu'), keras.layers.Dense(embed_dim),]
        )
#         self.layernorm_1 = keras.layers.LayerNormalization()
#         self.layernorm_2 = keras.layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention([inputs, inputs, inputs])
        proj_input = keras.layers.BatchNormalization()(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return keras.layers.BatchNormalization()(proj_input + proj_output)

def Transformer(
        input_shapes,
        output_size,
        feature_extractor,
        *args,
        **kwargs):
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = 32
    dense_dim = 16
    num_heads = 1
    video = keras.layers.Input(shape=input_shapes,name='video_input')
    encoded_frame = keras.layers.TimeDistributed(keras.layers.Lambda(lambda x: feature_extractor(x)))(video)
    
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(encoded_frame)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    x = keras.layers.Dropout(0.5)(x)
        
    # encoded_vid = keras.layers.Dense(8, activation='relu')(encoded_vid)
    outputs = keras.layers.Dense(output_size, activation='softmax')(x)
    
    model = keras.models.Model(inputs=[video],outputs=outputs)
    
    return model

In [12]:
def LRCNs(
        input_shapes,
        output_size,
        feature_extractor,
        hidden_state_num = 2,
        hidden_state_size = (16, 8),
        *args,
        **kwargs):
    feature_extractor.trainable = True
    video = keras.layers.Input(shape=input_shapes,name='video_input')
    encoded_frame = keras.layers.TimeDistributed(keras.layers.Lambda(lambda x: feature_extractor(x)))(video)
    
    for i in range(0, hidden_state_num - 1):
        encoded_frame = keras.layers.LSTM(hidden_state_size[i], return_sequences=True)(encoded_frame)
        
    encoded_vid = keras.layers.LSTM(hidden_state_size[hidden_state_num-1], return_sequences=False)(encoded_frame)
    encoded_vid = keras.layers.Dropout(0.3)(encoded_vid)
        
    # encoded_vid = keras.layers.Dense(8, activation='relu')(encoded_vid)
    outputs = keras.layers.Dense(output_size, activation='softmax')(encoded_vid)
    
    model = keras.models.Model(inputs=[video],outputs=outputs)
    
    return model

In [13]:
model = LRCNs((None, 6), 2, feature_extractor)
# model = Transformer((None, 6), 2, feature_extractor)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
video_input (InputLayer)     (None, None, 6)           0         
_________________________________________________________________
time_distributed (TimeDistri (None, None, 6)           0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 16)          1472      
_________________________________________________________________
lstm_1 (LSTM)                (None, 8)                 800       
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 2,290
Trainable params: 2,290
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"]
)

In [16]:
model.fit(train_x, train_y, shuffle=True,
      batch_size=50, epochs=100, validation_split=0.15,
      verbose=1)

Train on 1093 samples, validate on 194 samples
Epoch 1/100
1093/1093 [==============================] - 3s 3ms/sample - loss: 0.6808 - acc: 0.5590 - val_loss: 1.0032 - val_acc: 0.0000e+00
Epoch 2/100
1093/1093 [==============================] - 2s 2ms/sample - loss: 0.6056 - acc: 0.7072 - val_loss: 1.2838 - val_acc: 0.0000e+00
Epoch 3/100
1093/1093 [==============================] - 2s 2ms/sample - loss: 0.5908 - acc: 0.7072 - val_loss: 1.2017 - val_acc: 0.0000e+00
Epoch 4/100
1093/1093 [==============================] - 2s 2ms/sample - loss: 0.5656 - acc: 0.7072 - val_loss: 1.3482 - val_acc: 0.0000e+00
Epoch 5/100
1093/1093 [==============================] - 2s 2ms/sample - loss: 0.4707 - acc: 0.7548 - val_loss: 2.0661 - val_acc: 0.0206
Epoch 6/100
1093/1093 [==============================] - 2s 2ms/sample - loss: 0.3270 - acc: 0.8957 - val_loss: 2.5361 - val_acc: 0.0928
Epoch 7/100
1093/1093 [==============================] - 2s 2ms/sample - loss: 0.2442 - acc: 0.9442 - val_loss: 2.5

Epoch 60/100
1093/1093 [==============================] - 2s 2ms/sample - loss: 0.0207 - acc: 0.9918 - val_loss: 0.9731 - val_acc: 0.7526
Epoch 61/100
1093/1093 [==============================] - 2s 2ms/sample - loss: 0.0133 - acc: 0.9954 - val_loss: 0.5152 - val_acc: 0.8814
Epoch 62/100
 900/1093 [=======================>......] - ETA: 0s - loss: 0.0437 - acc: 0.9922

KeyboardInterrupt: 